In [ ]:
import os
import tweepy as tw, pandas as pd
import re
from geopy.geocoders import GoogleV3


In [ ]:
pd.set_option('max_colwidth', 500)

In [ ]:
geoloc = Nominatim(user_agent=__name__)
loc = geoloc.geocode("La riviera en pedregal")
print(loc)

### Connecting to API

In [ ]:
consumer_key = 'A5YS1UVZEbiIZdbiUKKageZKe'
consumer_secret = 'z13JcgAG0v8ratKmMVPrsLGNiDkM7XnyG2AWvTmf5agebGnodK'
access_token = '1344702616186589185-HG0juN3inE9vk3M4X52vFQ9ewWoNvj'
access_token_secret = 't9TOMRE3XzEdqj1hbmeqHlnMiddMVyuBO7dY5RvvKDmMZ'

In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

### Obtaining data

In [ ]:
# Define the word to seach and date
search_word = "(from:traficocpanama) OR (@traficocpanama) -filter:replies -filter:retweets"
date_since = "2021-01-01"

# Collect tweets
tweets = tw.Cursor(api.search,
                q=search_word,
                lang='es',
                since=date_since,
                tweet_mode='extended').items()

all_info = [[t] for t in tweets]

### Connection to database

In [ ]:
import psycopg2 as ps
try:
    conn = ps.connect(database='twitter', user='admin', password='admin123')
    print('Connection succesful.')
except: print('Error connecting to database.')

### Tables creation

In [ ]:
commands = (# Table User
            '''Create Table TwitterUser(USER_ID BIGINT PRIMARY KEY,
                                        USER_NAME TEXT, USER_LOCATION TEXT);''',
            # Table Tweet
            '''Create Table TwitterTweet(TWEET_ID BIGINT PRIMARY KEY,
                                         TWEET_USER_ID BIGINT, TWEET_TEXT TEXT,
                                         TWEET_CREATED TIMESTAMP WITH TIME ZONE,
                                         TWEET_COORDINATES TEXT,TWEET_PLACE TEXT,
                                         CONSTRAINT FK_USER FOREIGN KEY(TWEET_USER_ID)
                                                 REFERENCES TwitterUser(USER_ID));''')
cursor = conn.cursor()

try:
    for command in commands:
        # create tables
        cursor.execute(command)
    print('Tables created successfully.')
except Exception as e: print(e)

conn.commit()
cursor.close()

### Insert data into tables function

In [ ]:
def insert_into_db(conn, userid, username, userloc, twtid, text, twtdate, twtcoods, twtplace):
    cursor = conn.cursor()

    try:
        # insert user
        command = '''INSERT INTO TWITTERUSER(USER_ID, USER_NAME, USER_LOCATION)
                    VALUES (%s, %s, %s) ON CONFLICT (USER_ID) DO NOTHING;'''
        cursor.execute(command, (userid, username, userloc))

        # insert tweet
        command = '''INSERT INTO TWITTERTWEET(TWEET_ID, TWEET_USER_ID, TWEET_TEXT,
                    TWEET_CREATED, TWEET_COORDINATES, TWEET_PLACE)
                    VALUES (%s, %s, %s, %s, %s, %s);'''
        cursor.execute(command, (twtid, userid, text, twtdate, twtcoods, twtplace))
    except Exception as e: print(e)

    # commit changes and close cursor
    conn.commit()
    cursor.close()

### Inserting twt data into database

In [ ]:
t = all_info[5][0]
insert_into_db(conn, t.user.id, t.user.screen_name, t.user.location, \
        t.id, t.full_text, t.created_at, t.coordinates, t.place)

In [ ]:
# i=0
for info in all_info:
    t = info[0]
    insert_into_db(conn, t.user.id, t.user.screen_name, t.user.location, \
        t.id, t.full_text, t.created_at, \
            t.coordinates if t.coordinates!=None else "", \
                t.place if t.place!=None else "")